## Numerical Analysis - Fall semester 2024

# Serie 11 - Ordinary differential equations

Package imports.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp

<hr style="clear:both">

### Explicit methods for population model

Let us consider an ordinary differential equation of the form

$$
\begin{cases}
  \frac{\mathrm{d}u(t)}{\mathrm{d}t} = f(t, u(t)), & t \in (0, T],  \\
  u(0)=u_0.
\end{cases}
$$

We want to approximate $u(t_n)$ at the discrete time-steps $t_n = n\Delta t$, $n = 0, 1, 2, \ldots,N$, where $\Delta t = T/N$ is the step size and $N$ the number of time-steps.

<div class="alert alert-success">
    
**Exercise 1:** Complete the function `forward_euler` which implements the forward Euler method to compute the approximations $u^{n} \approx u(t_n)$ for $n = 0, 1, 2, \dots, N$.
</div>

In [ ]:
def forward_euler(f, u_0, T, N):
    ### BEGIN SOLUTION
    dt = T / N
    u = np.zeros(N + 1)
    u[0] = u_0
    for n in range(N):
        u[n + 1] = u[n] + dt * f(n * dt, u[n])
    ### END SOLUTION
    return u

<div class="alert alert-success">
    
**Exercise 2:** Complete the function `heun` which implements the Heun method to compute the approximations $u^{n} \approx u(t_n)$ for $n = 0, 1, 2, \dots, N$.
</div>

In [ ]:
def heun(f, u_0, T, N):
    ### BEGIN SOLUTION
    dt = T / N
    u = np.zeros(N + 1)
    u[0] = u_0
    for n in range(N):
        u[n + 1] = u[n] + dt / 2 * f(n * dt, u[n]) + dt / 2 * f((n + 1) * dt, u[n] + dt * f(n * dt, u[n])) 
    ### END SOLUTION
    return u

Let us consider a population of $u$ individuals in an environment
where at most $u_{\max}=1000$ individuals can coexist. We suppose that initially
the number of individuals is $u_0 = 100$ and that the growth factor equals
a constant $C=2/15$. The considered model for the evolution of the
population is the following:

$$
\left\{\begin{array}{ll}
  \frac{\mathrm{d}u(t)}{\mathrm{d}t}=Cu(t)\left(1-\frac{u(t)}{u_{\max}}\right), & t \in (0,100],  \\
  u(0)=u_0. & 
\end{array}
\right.
$$

<div class="alert alert-success">
    
**Exercise 3:** Use the two functions you have implemented above to compute the approximate solution $u^n$, $n=0,1,...,N$ for $N=20$ time-steps and plot on the same graph the obtained numerical solutions in terms of time. Compare the obtained approximations for both forward Euler and Heun with the exact solution given by
  $$
  u(t) = \left( \frac{1-e^{-Ct}}{u_{\max}} + \frac{e^{-Ct}}{u^0} \right)^{-1}.
  $$
  Which method gives a better approximation?
</div>


In [ ]:
### BEGIN SOLUTION

# RHS of ODE
def f(t, u, u_max=1000, C=2/15):
    return C * u * (1 - u / u_max)

# Exact solution
def u(t, u0, u_max=1000, C=2/15):
    return 1 / ((1 - np.exp(- C * t)) / u_max + np.exp(- C *  t) / u0)

# Parameters
T = 100
u_0 = 100
N = 20

# Solve ODEs with the two methods
u_forward_euler = forward_euler(f, u_0, T, N)
u_heun = heun(f, u_0, T, N)

# Determine exact solution
t = np.linspace(0, T, N + 1)
u_exact = u(t, u_0)

# Visualize the results
plt.plot(t, u_forward_euler, label="forward Euler")
plt.plot(t, u_heun, label="Heun")
plt.plot(t, u_exact, color="black", linestyle="--", label="exact")
plt.ylabel(r"$u(t)$")
plt.xlabel(r"$t$")
plt.grid()
plt.legend()
plt.show()

# Comparing the obtained approximations with the exact solution, we notice that
# Heun gives a better approximation of the true solution than forward Euler. 
# This is expected, because Heun's method is more accurate than forward Euler
# as it uses a corrector step to refine the solution by averaging slopes and
# the forward Euler method tends to under- or overshoot, especially for larger
# time steps, due to its simple one-step approximation.

### END SOLUTION

<div class="alert alert-success">
    
**Exercise 4:** Repeat the last exercise using $N=2000$ time-steps. Is the obtained approximation with $N=2000$ better than the one obtained with $N=20$?
</div>



In [ ]:
### BEGIN SOLUTION

# Parameters
T = 100
u_0 = 100
N = 2000

# Solve ODEs with the two methods
u_forward_euler = forward_euler(f, u_0, T, N)
u_heun = heun(f, u_0, T, N)

# Determine exact solution
t = np.linspace(0, T, N + 1)
u_exact = u(t, u_0)

# Visualize the results
plt.plot(t, u_forward_euler, label="forward Euler")
plt.plot(t, u_heun, label="Heun")
plt.plot(t, u_exact, color="black", linestyle="--", label="exact")
plt.ylabel(r"$u(t)$")
plt.xlabel(r"$t$")
plt.grid()
plt.legend()
plt.show()

# The lines for the forward Euler, Heun's method, and the exact solution are almost
# indistinguishable from each other. This suggests that the numerical methods with
# a large number of time steps (N = 2000) closely approximate the exact solution.
# The fact that the numerical solutions align so well with the exact solution
# indicates that the step size (Δt = T/N) is sufficiently small to reduce numerical
# error significantly.

### END SOLUTION

<hr style="clear:both">

### Order of convergence

Let us consider the following differential equation

$$
\begin{cases}
\frac{\mathrm{d}u(t)}{\mathrm{d}t} = \sqrt{u(t)}\frac{1}{1+t^2}, \quad t \in (0,20],\\[3mm]
u(0)=1.
\end{cases}
$$

We again want to approximate $u(t_n)$ at the discrete time-steps $t_n = n\Delta t$, $n = 0, 1, 2, \ldots,N$, where $\Delta t = T/N$ is the step size and $N$ the number of time-steps.

We provide you with the function `backward_euler` which implements the backward Euler method.

In [ ]:
def backward_euler(f, u_0, T, N):
    dt = T / N
    u = np.zeros(N + 1)
    u[0] = u_0
    for n in range(N):
        def g(x):
            return x - dt * f((n + 1) * dt, x) - u[n]
        u[n + 1] = sp.optimize.root(g, u[n]).x.item()
    return u

<div class="alert alert-success">
    
**Exercise 5:** Solve the equation using the `backward_euler` function which implements the backward Euler method for $N = 50, 100, 200, 400$ and visualize the obtained solution for each value of $N$. 
</div>

In [ ]:
### BEGIN SOLUTION 

# RHS of ODE
def f(t, u):
    return np.sqrt(u) / (1 + t ** 2)

# Parameters
T = 20
u_0 = 1

# Compute and plot solution for each N
N_list = [50, 100, 200, 400]
for N in N_list:
    u_backward_euler = backward_euler(f, u_0, T, N)
    t = np.linspace(0, T, N + 1)
    plt.plot(t, u_backward_euler, label=r"$N = {:d}$".format(N))
plt.ylabel(r"$u(t)$")
plt.xlabel(r"$t$")
plt.grid()
plt.legend()
plt.show()

### END SOLUTION

<div class="alert alert-success">
    
**Exercise 6:** The exact solution of the differential equation from the start of this section is
$$ u(t) = ( 1 + 0.5 \ {\rm atan}(t) )^2. $$
Compute the error
$$E_{\Delta t} = \max_{0\leq n\leq N} |u(t_n) - u^n|$$ 
of the approximations $u^n$ for the backward Euler method for $N = 50, 100, 200, 400$. Check graphically that the order of convergence of the method is $1$ by plotting the errors $E_{\Delta t}$ against the step-sizes $\Delta t$ which are used for each $N$.
</div>


In [ ]:
### BEGIN SOLUTION 

def u(t):
    return (1 + np.arctan(t) / 2) ** 2

# Parameters
T = 20
u_0 = 1

N_list = np.array([50, 100, 200, 400])
dt_list = T / N_list
error_list = []
for N in N_list:
    u_backward_euler = backward_euler(f, u_0, T, N)
    t = np.linspace(0, T, N + 1)
    u_exact = u(t)
    error_list.append(np.max(np.abs(u_exact - u_backward_euler)))

plt.loglog(dt_list, error_list, label=r"$E_{\Delta t}$")
plt.loglog(dt_list, dt_list, color="black", linestyle="--", label=r"$\Delta t$")
plt.xlabel(r"$\Delta t$")
plt.grid()
plt.legend()
plt.show()

### END SOLUTION 

<div class="alert alert-success">
    
**Exercise 7:** Repeat the previous exercise for the forward Euler and Heun methods using the functions `forward_euler` and `heun` you have implented in the first section of this notebook. What order of convergence do you observe for the two methods?
</div>

In [ ]:
### BEGIN SOLUTION 

def u(t):
    return (1 + np.arctan(t) / 2) ** 2

# Parameters
T = 20
u_0 = 1

N_list = np.array([50, 100, 200, 400])
dt_list = T / N_list
error_list_forward_euler = []
error_list_heun = []
for N in N_list:
    u_forward_euler = forward_euler(f, u_0, T, N)
    u_heun = heun(f, u_0, T, N)
    t = np.linspace(0, T, N + 1)
    u_exact = u(t)
    error_list_forward_euler.append(np.max(np.abs(u_exact - u_forward_euler)))
    error_list_heun.append(np.max(np.abs(u_exact - u_heun)))

plt.loglog(dt_list, error_list_forward_euler, label=r"$E_{\Delta t}$ for forward Euler")
plt.loglog(dt_list, error_list_heun, label=r"$E_{\Delta t}$ for Heun")
plt.loglog(dt_list, dt_list, color="black", linestyle="--", label=r"$\Delta t$")
plt.loglog(dt_list, dt_list ** 2, color="black", linestyle=":", label=r"$\Delta t^2$")
plt.xlabel(r"$\Delta t$")
plt.grid()
plt.legend()
plt.show()

# The figure shows the behavior of the error E_Δt in terms of Δt for the three methods.
# We see that the forward Euler is of first order, whereas Heun is of second order.

### END SOLUTION 

<hr style="clear:both">

## The end

Perfect! This was the eleventh exercise notebook. Just two more to do.